In [ ]:
!pip install cryptography
!pip install sha3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import libraries
from secrets import randbits
from sympy import Matrix
from sympy.abc import x
import random
import hashlib
import sha3
from functools import reduce
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import os
from cryptography.fernet import Fernet

## Architecture Parameters

In [ ]:
# defining the nodes
master = {}
racks_c = {}

In [ ]:
# architecture parameters
a = 1
b = 15
replication_factor = 5
rack_system = {}
public_keys = {}
private_keys = {}

In [ ]:
# keys for racks
consts = {}
for i in range(1, 16, 1):
  consts[i] = [randbits(16) for _ in range(256)]

In [ ]:
# rack intelligence system
def get_racks():
  racks = random.sample(range(a, b+1), replication_factor)
  print('racks generated')
  return racks

## CRYSTALS Kyber & Hashing

In [ ]:
def generate_global_key_pair():

  def get_random_poly():
      """Generate a random polynomial."""
      coeffs = [randbits(16) for _ in range(256)]
      return coeffs

  def evaluate_poly(poly, point):
      """Evaluate a polynomial at a given point."""
      return sum(coeff * point ** i for i, coeff in enumerate(poly))

  def generate_key_pair():
      """Generate a public/private key pair."""
      # Generate a random polynomial
      private_key = get_random_poly()

      # Evaluate the polynomial at 256 points to get the public key
      public_key = [evaluate_poly(private_key, point) % 3329 for point in range(256)]

      return public_key, private_key

  public_key, private_key = generate_key_pair()
  print('global keys generated')
  return public_key, private_key

In [ ]:
def generate_rack_key_pair(racks):
  """Generate a public/private key pair."""

  def evaluate_poly(poly, point):
    """Evaluate a polynomial at a given point."""
    return sum(coeff * point ** i for i, coeff in enumerate(poly))

  def get_private_key(racks):
    """Generate the private key based on racks"""
    private_key = [0 for _ in range(256)]
    for rack in racks:
      private_key = [(a+b)%(2**16) for a,b in zip(private_key, consts[rack])]
    return private_key

  # Generate a random polynomial
  private_key = get_private_key(racks)

  # Evaluate the polynomial at 256 points to get the public key
  public_key = [evaluate_poly(private_key, point) % 3329 for point in range(256)]

  # Return the public key and the polynomial (as the private key)
  print('rack keys generated')
  return public_key, private_key

In [ ]:
def get_nonce(racks, curr_rack):
  racks = [rack for rack in racks if rack!=curr_rack]
  nonce = [sum(x) for x in zip(racks)]
  nonce = [x % (2**16) for x in nonce]
  print('rack nonce generated')
  return nonce

In [ ]:
def get_global_nonce(racks):
  keys = []
  for rack in racks:
    keys.append(consts[rack])
  nonce = [sum(x) for x in zip(*keys)]
  fnonce = [x % (2**16) for x in nonce]
  print('global nonce generated')
  return fnonce

In [ ]:
def get_rackcrypt(racks):
  randsums = [(sum(consts[rack]) % (2**16)) for rack in racks]
  product = reduce(lambda x, y: x*y, randsums) % (2**16)
  print('rackcrypt generated')
  return product

In [ ]:
def get_sha(msg):
  return hashlib.sha256(msg).hexdigest()

In [ ]:
def encapsulate_key(public_key, nonce):
    """Encapsulate a shared secret using the public key."""

    def evaluate_poly(poly, point):
      """Evaluate a polynomial at a given point."""
      return sum(coeff * point ** i for i, coeff in enumerate(poly))

    # Evaluate the polynomial at 256 points to get the shared secret
    shared_secret = [evaluate_poly(nonce, point) % 3329 for point in range(256)]

    # Evaluate the public key at each point and add the corresponding coefficient of the polynomial
    error_vector = [(evaluate_poly(public_key, point) + s) % 3329 for point, s in enumerate(shared_secret)]

    # Return the shared secret and the error vector
    print('key encapsulated')
    return shared_secret, error_vector

In [ ]:
def encode(public_key, nonce, message):
  key, _= encapsulate_key(public_key, nonce)
  key = (bin(sum(key) % (2**16))[2:])
  encrypted_data = int(key) | int(message)
  print('aes done')
  return encrypted_data

In [ ]:
def update_master(uid):
  rackcrypt = get_rackcrypt(rack_system[uid])
  sha1 = get_sha(bin(rackcrypt)[2:].encode('utf-8'))
  sha2 = get_sha(bin(int(sha1, 16))[2:].encode('utf-8'))
  hash = int(str(bin(int(sha2, 16))[2:])+str(bin(rackcrypt)[2:])+str(bin(int(sha1, 16))[2:]))
  encrypted = encode(global_public_key, global_nonce, hash)
  master[uid] = encrypted
  print('master updated')

In [ ]:
def store(uid, message):
  racks = rack_system[uid]
  sha1 = get_sha(bin(message)[2:].encode('utf-8'))
  sha2 = get_sha(bin(int(sha1, 16))[2:].encode('utf-8'))
  hash = int(str(bin(int(sha2, 16))[2:])+str(bin(message)[2:])+str(bin(int(sha1, 16))[2:]))

  for rack in racks:
    nonce = get_nonce(racks, rack)
    public_key = public_keys[uid]
    encrypted = encode(public_key, nonce, hash)
    racks_c['c{}'.format(rack)] = encrypted
  print('stored')

## User End

In [ ]:
# user's racks
def set_user_racks(uid):
  returning = True
  if rack_system.get(uid) is None:
    rack_system[uid] = get_racks()
    returning = False
  return returning, rack_system[uid]
  print('racks set')

# user's keys
def set_keys(uid):
  keys =  generate_rack_key_pair(rack_system[uid])
  public_keys[uid] = keys[0]
  private_keys[uid] = keys[1]
  print('keys set')

# generate global keys
global_public_key, global_private_key = generate_global_key_pair()
global_nonce = get_global_nonce(list(range(1, 16, 1)))

global keys generated
global nonce generated


In [ ]:
def main():

  # input user details
  uid = int(input('Enter User ID: '))

  # get user's racks
  returning, racks = set_user_racks(uid)
  if returning == False:
    set_keys(uid)
    update_master(uid)


  msg = input(('Enter message: '))
  bitmsg = ''
  for c in msg:
    bitmsg += bin(ord(c))[2:].zfill(8)
  sha1 = get_sha(bin(int(bitmsg))[2:].encode('utf-8'))
  sha2 = get_sha(bin(int(sha1, 16))[2:].encode('utf-8'))
  message = int(str(bin(int(sha2, 16))[2:])+
                str(bin(int(bitmsg))[2:])+
                str(bin(int(sha1, 16))[2:]))

  store(uid, message)

In [ ]:
main()

Enter User ID: 0
racks generated
rack keys generated
keys set
rackcrypt generated
key encapsulated
aes done
master updated
Enter message: hello
rack nonce generated
key encapsulated
aes done
rack nonce generated
key encapsulated
aes done
rack nonce generated
key encapsulated
aes done
rack nonce generated
key encapsulated
aes done
rack nonce generated
key encapsulated
aes done
stored


In [ ]:
rack_system

{0: [3, 14, 5, 7, 1]}

In [ ]:
master

{0: 101101111000011011001000011101011100111010100010111010000110011010101101101101111001010001110001101111110010000010011111100110111110111101110110011100011010100000100010101101100101000011111010110111100101110100111101110110101000100101110101110011110010110000101010000010010101001100001111100100110010100111010000000000010111101011100011110011010000101010111001010011101100000101001100111011010000111001010100000110101111010100110011000000010001100011001100110000111000010111010100010111000011110110001000110117130803834359}

In [ ]:
racks_c

{'c3': 100110101000110110111111000000011101110001110100100001100000101011010111100011100000011010100011110011000111110110001110110010110011011000110010001001000111110000101110000101111100110111011010100000010100001010101001110111100000010010111010101011001110111110111000101101001010011111000110111010111100001000111001101011011010111011010010100101110110010110111010011000000001111101100100011001110000010100000110010100001011011010100000101110001001100100110100011010011011101110011100110101001010110010110000101010111010101100000111111001001111100010100011101011101001101101111011010000111111001110101011101111010010101111001001010111000110011010111010001010110110101011101101010100011000110101011111011011111111011110001111000001100110101111100111001110100010010010110011010001001100111100000101111110101010110111011100101001101101011000011101000011111010001101101101111111011011100010101010110100101000100111001110010010101110000001000001100001101011101000011001001110010011110000001001000001001